In [1]:
import sys
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage

class Process(object):
    numKeypoint = 17
    keypointsName = ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 
                    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 
                    'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 
                    'right_knee', 'left_ankle', 'right_ankle']

    skeleton = [[15, 13], [13, 11], [16, 14], 
                [14, 12], [11, 12], [5, 11], 
                [6, 12], [5, 6], [5, 7], [6, 8], 
                [7, 9], [8, 10], [1, 2], [0, 1], 
                [0, 2], [1, 3], [2, 4], [3, 5], [4, 6]]
    
    trainAnnoPath = 'COCO/annotations/person_keypoints_train2017.json'
    valAnnoPath = 'COCO/annotations/person_keypoints_val2017.json'
    
    def loadData(self, setType):
        if setType == 'train':
            coco = COCO(self.trainAnnoPath)
        elif setType == 'val':
            coco = COCO(self.valAnnoPath)
            
        Data = []
        for aid in coco.anns.keys():
            ann =  coco.anns[aid]
            keypoints = ann['keypoints']
            
            if  keypoints.count(0) == 0: 
                imageName = 'COCO/' + setType + '2017/' + coco.imgs[ann['image_id']]['file_name']

                if (ann['image_id'] not in coco.imgs) or ann['iscrowd'] or (np.sum(keypoints[2::3]) == 0) or (ann['num_keypoints'] == 0):
                    continue

                data = dict(image_id = ann['image_id'], imgpath = imageName, bbox=ann['bbox'], keypoints=keypoints)
                Data.append(data)
            else:
                continue
            
        return Data
    
    def imagesToNdarry(self, data): 
        imgPath = []
        i = 0
        for i in range(len(data)):
            img = cv2.imread(data[i]['imgpath'])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgPath.append(img)
            
        return np.array(imgPath)
    
    def splitKeypoints(self, data):
        kpsList = []
        visibiltyList = []
        for ann in data:
            keypoints = np.array(ann['keypoints']).astype(int)
            keypoints =  keypoints.reshape(17,3)
            keypoints = self.reshapeKeypoints(keypoints)
            kpsList.append(keypoints['kps'])
            visibiltyList.append(keypoints['visibility'])

        return kpsList, visibiltyList
    
    def reshapeKeypoints(self, keypoints):
        keypoint = []
        visibility = []

        for kps in keypoints: 
            x,y,v = kps 
            keypoint.append(Keypoint(x,y))
            visibility.append(v)
            
        res = dict()
        res['kps'] = keypoint; res['visibility'] = visibility
        return res
    
#     def augmentKeypoints(self, keypoints, shape):
#         augKpsList = []
#         for kps in keypoints:
#             augKps = self.augmentation.augment_keypoints(KeypointsOnImage(kps, shape))
#             augKpsList.append(augKps)
#         return augKpsList

    def augmentationOnAll(self, images, kps):
        seq = iaa.Sequential([
            iaa.Resize(112),            
        ])

        imgAugList = []
        kpsAugList = []
        
        i = 0 
        for i in range(len(images)):
            imgAug, kpsAug = seq(image=images[i], keypoints=KeypointsOnImage(kps[i],shape=images[i].shape))
            imgAugList.append(imgAug)
            kpsAugList.append(kpsAug)
            
        return np.array(imgAugList), kpsAugList

    def mergeKpsVisibility(self, augKps, visibility, kpShape, MergeShape):
        kpsList = [] 
        i = 0
        for i in range(len(augKps)):
            for keypoint in augKps[i]:
                x,y = keypoint.x, keypoint.y
                kpsList.append(x)
                kpsList.append(y)
        kpsList = np.array(kpsList).reshape(kpShape)
        merge = np.dstack((kpsList, visibility))
        merge = merge.reshape(MergeShape)
        return merge

processing = Process()

In [ ]:
# fetchedData = processing.loadData('train')

In [ ]:
# print(len(fetchedData))
# batch1 = fetchedData[:2500]
# there are in total of 18 batches but our pc can't handle it. (5551 * 18)

In [ ]:
# images = processing.imagesToNdarry(batch1)
# allKeypoints, allVisibility = processing.getAllKeypoints(batch1)
# print(len(allKeypoints), len(images))

In [ ]:
# x_train, kps = processing.augmentationOnAll(images, allKeypoints)
# y_train = processing.mergeKpsVisibility(kps, allVisibility, (2500, 17,2), (2500, 51))

In [ ]:
# y_train.shape

In [ ]:
# #check if x_train is 4dArray
# print(x_train.shape)

# #check resized keypoints value 
# print(kps[1])